In [122]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd

import re
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from nltk.corpus import stopwords
from sklearn import metrics
from sklearn.metrics import accuracy_score, f1_score, average_precision_score, recall_score
from nltk.tokenize import word_tokenize
from string import punctuation
from nltk.stem import WordNetLemmatizer 
lemma = WordNetLemmatizer()
from wordcloud import WordCloud
from nltk import FreqDist
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

In [123]:
#Load the dataset
df = pd.read_csv('blogtext.csv', nrows= 5000)


In [124]:
df.head()

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,..."
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...


In [125]:
df.shape

(5000, 7)

In [126]:
df.isnull().sum()

id        0
gender    0
age       0
topic     0
sign      0
date      0
text      0
dtype: int64

In [127]:
df = df.head(3000)

In [128]:
# Selecting the alphabets
df.text = df.text.apply(lambda x: re.sub('[^A-Za-z]+', ' ', x))



In [129]:
# Converting to Lower Case
df.text = df.text.apply(lambda x: x.lower())



In [130]:
# Strip the Unwanted Keywords
df.text = df.text.apply(lambda x: x.strip())

In [131]:
#mergeall the label columns together
df1=df[['gender', 'age', 'topic', 'sign']]

In [132]:
df1['age']=df1['age'].astype('str')

In [133]:
m=[]                              
for i in range(df1.shape[0]):
    g=[]                          
    for j in range(df1.shape[1]):
        g.append(df1.iloc[i][j])
    m.append(g)

In [134]:
df['labels']=m

In [135]:
df2 = df[['text', 'labels']]
df2.head(5)

,text,labels
0,info has been found pages and mb of pdf files ...,"[male, 15, Student, Leo]"
1,these are the team members drewes van der laag...,"[male, 15, Student, Leo]"
2,in het kader van kernfusie op aarde maak je ei...,"[male, 15, Student, Leo]"
3,testing testing,"[male, 15, Student, Leo]"
4,thanks to yahoo s toolbar i can now capture th...,"[male, 33, InvestmentBanking, Aquarius]"


In [136]:
df2['labels'].astype('str').value_counts()

['male', '35', 'Technology', 'Aries']               1607
['female', '24', 'indUnk', 'Scorpio']                191
['female', '15', 'Student', 'Libra']                 131
['male', '24', 'Engineering', 'Libra']               119
['female', '27', 'Education', 'Aquarius']             85
['male', '17', 'Sports-Recreation', 'Capricorn']      75
['male', '33', 'InvestmentBanking', 'Aquarius']       70
['female', '15', 'Student', 'Aquarius']               62
['male', '23', 'indUnk', 'Sagittarius']               49
['male', '25', 'Non-Profit', 'Cancer']                46
['male', '14', 'Student', 'Scorpio']                  46
['male', '26', 'indUnk', 'Leo']                       35
['male', '15', 'Student', 'Aquarius']                 34
['female', '25', 'Student', 'Taurus']                 33
['male', '17', 'Student', 'Sagittarius']              32
['male', '39', 'Education', 'Virgo']                  32
['male', '15', 'Science', 'Libra']                    29
['female', '15', 'indUnk', 'Ari

In [137]:
#split the data into training and testing
X_train, X_test, y_train, y_test = train_test_split(df.text.values, df.labels.values, test_size=0.20, random_state=40)

In [139]:
#Vectorize the features
vectorizer = CountVectorizer(binary=True, ngram_range=(1, 2))
X_train_r = vectorizer.fit_transform(X_train)
X_test_r = vectorizer.transform(X_test)

In [140]:
vectorizer.get_feature_names()[:10]

['aa',
 'aa compared',
 'aa nice',
 'aa or',
 'aaaaack',
 'aaaah',
 'aaaah the',
 'aaaahh',
 'aaah',
 'aaah what']

In [141]:
label_counts = dict()

for labels in df.labels.values:
    for label in labels:
        if label in label_counts:
            label_counts[label] += 1
        else:
            label_counts[label] = 1

In [142]:
#Transform the labels
mlb = MultiLabelBinarizer(classes=sorted(dictionary.keys()))
y_train_y = mlb.fit_transform(y_train)
y_test_y = mlb.transform(y_test)

In [143]:
# Choose a classifier

clf = LogisticRegression(solver='lbfgs')
clf = OneVsRestClassifier(clf)

In [145]:
#Fit the classifier, make predictions and get the accuracy 
clf.fit(X_train_r, y_train_y)

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [153]:
predict_N = clf.predict(X_test_r)
predict_scores = clf.decision_function(X_test_r)

In [154]:
pred_inverse = mlb.inverse_transform(predict_N)
y_test_inverse = mlb.inverse_transform(y_test_y)

In [156]:
def print_evaluation_scores(y_val, predicted):
    print('Accuracy score: ', accuracy_score(y_val, predicted))
    print('F1 score: ', f1_score(y_val, predicted, average='micro'))
    print('Average precision score: ', average_precision_score(y_val, predicted, average='micro'))
    print('Average recall score: ', recall_score(y_val, predicted, average='micro'))

In [159]:
print('Bag-of-words')
print_evaluation_scores(y_test_y, predict_N)

Bag-of-words
Accuracy score:  0.5466666666666666
F1 score:  0.7460093896713615
Average precision score:  0.5906493677817602
Average recall score:  0.6620833333333334
